<a href="https://colab.research.google.com/github/bukhariaujih/KAU-training/blob/master/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !git clone https://github.com/bukhariaujih/KAU-training.git

fatal: destination path 'KAU-training' already exists and is not an empty directory.


In [13]:
# Mount data
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Install depedencies packages

# !pip install scikit-plot
# !pip install pillow
# !pip install scipy
# !pip install scikit-learn
# !apt-get upgrade -y
# !apt-get update
# !apt-get install -y libsm6 libxext6 libxrender-dev
# !pip install opencv-python==3.4.2.17

In [0]:
# Initialization
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, confusion_matrix

import cv2

import scikitplot as skplt

import os, sys, glob

import matplotlib.pyplot as plt

import numpy as np

%matplotlib inline

In [0]:
# Routine function
def check_dir(path):
    if os.path.exists(path):
        print("Dir already exist")
    
    elif sys.platform == 'win32':
        os.system('mkdir ' + path)
    
    else:
        os.system('mkdir -p ' + path)
        print('New Path created : ', path)
    return 
    
def add_new_last_layer(base_model, nb_classes):
    print('Add last layer to the convnet..')
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='relu')(x)  
    predictions = Dense(nb_classes, activation='softmax')(x) 
    model = Model(inputs=base_model.input, 
                  outputs=predictions) 
    return model

def get_im(path):
    img = cv2.imread(path, 0)
    resized = cv2.resize(img, (224, 224))
    return resized

def load_train():
    nb_classes = 2
    img_rows, img_cols = 224, 224
    X_train = []
    y_train = []

    root = './drive/My Drive/training_1/Data/image'
    path= root + '/dme/*.png'
    files = glob.glob(path)
    for fl in files:
        img = get_im(fl)
        X_train.append(img)
        y_train.append(1)

    path=root + '/normal/*.png'
    files = glob.glob(path)
    for fl in files:
        img = get_im(fl)
        X_train.append(img)
        y_train.append(0)

    X_train = np.array(X_train, dtype=np.uint8)
    y_train = np.array(y_train, dtype=np.uint8)
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    # train_data = train_data.transpose((0, 3, 1, 2))
    y_train = to_categorical(y_train, nb_classes)
    X_train = X_train.astype('float32')
    X_train /= 255
    print('Train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')

    return X_train, y_train

def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, 
                                                        target, 
                                                        test_size=test_size, 
                                                        random_state=random_state)
    print('Split train: ', len(X_train))
    print('Split valid: ', len(X_test))
    return X_train, X_test, y_train, y_test

In [27]:
# Load training data
train_data, train_target = load_train()

KeyboardInterrupt: ignored

In [0]:
# Split training data 
X_train, X_test, Y_train, Y_test = split_validation_set(train_data, 
                                                        train_target, 
                                                        0.2)

In [0]:
# Build base network
base_model = VGG19(include_top=False, 
                   input_shape=(224,224,1), 
                   weights=None)
model_name = 'VGG19'

base_model.summary()

In [0]:
# Add the new last layer to the network 
nb_classes = 2
model = add_new_last_layer(base_model, nb_classes)

learning_rate = 0.0001
decay_rate = learning_rate / 100
momentum = 0.8
SGD = optimizers.SGD(lr=learning_rate, 
                     momentum=momentum, 
                     decay=decay_rate, nesterov=False)

model.compile(optimizer=RMSprop(lr=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [0]:
# Prepare tensorboard
TB_LOG = './output/logs_tb'
check_dir(TB_LOG)

tensorboard = TensorBoard(log_dir=TB_LOG,
                          histogram_freq=0,
                          batch_size=32,
                          write_graph=True,
                          write_grads=False,
                          write_images=False,
                          embeddings_freq=0,
                          embeddings_layer_names=None,
                          embeddings_metadata=None)

In [0]:
# Train and save network
history = model.fit(X_train, Y_train, 
                    batch_size=32, 
                    epochs=5, 
                    verbose=1, 
                    validation_data=(X_test, Y_test))
model.save('./output/my_model.h5') 
del model 

In [0]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
# Load trained network
model = load_model('./output/my_model.h5')

In [0]:
# Do prediction on test data
Y_pred=model.predict([X_test])

In [0]:
# Performance report
target_names = ["normal", "dme"]
print(classification_report(np.argmax(Y_test, axis=1), 
                            np.argmax(Y_pred, axis=1), 
                            target_names=target_names))

In [0]:
# Plot roc curve
skplt.metrics.plot_roc_curve(np.argmax(Y_test,axis=1), Y_pred)
plt.show()

In [0]:
# Show confusion matrix
confusion_matrix(np.argmax(Y_test,axis=1), 
                 np.argmax(Y_pred,axis=1))
model.metric()